In [1]:
# Import dependencies
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
zipcodes = pd.read_csv('C:/Anaconda/US_zipcodes.csv')

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'C:/Anaconda/webscrapes/chromedriver'}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('log-level=3')
browser = Browser('chrome', **executable_path, wait_time=3,headless=True,options=chrome_options)

In [3]:
# Visit Trane Dealer Locator site
url = 'https://www.trane.com/residential/en/dealers.html'
browser.visit(url)
html = browser.html
trane_soup = BeautifulSoup(html, 'html.parser')

In [4]:
dealer_links=[]
for i in trane_soup.find_all(class_="dealer-directory__link"):
    dealer_links.append(i.get('href'))

In [19]:
len(dealer_links)

1859

In [ ]:
for i in dealer_links:
    print(i)

In [20]:
import time
start_time = time.time()

df = pd.DataFrame()
for dealer in dealer_links[0]:
    try:
        browser.visit(dealer)
        html = browser.html
        dealer_soup = BeautifulSoup(html, 'html.parser')
        
        name = dealer_soup.find(class_="headline headline--uppercase headline--dark").get_text(strip=True)
        try:
            street = dealer_soup.find(class_="address_st").get_text(strip=True)
        except:
            pass
        city_state_zip = dealer_soup.find(class_="address_city-state").get_text(strip=True)
        city = city_state_zip.split(',')[0]
        state = re.findall('(?<!\').+(?=\s)', city_state_zip.split(',')[1] )[0].strip()
        zipcode = city_state_zip.split(',')[1][-6:].strip()
        if any(c.isalpha() for c in zipcode) == True:
            country='Canada'
        else:
            country='United States'
        phone =  dealer_soup.find(class_="ss-phone").next_element.get_text(strip=True)
        lat = ''
        long = ''
        website = ''
        brand = 'Trane'
        
    except:
        pass
    dealer_df = pd.DataFrame({'name':name,
                             'street':street,
                             'city':city,
                             'state':state,
                             'zipcode':zipcode,
                             'country':country,
                             'phone':phone,
                             'lat':lat,
                             'long':long,
                             'website':website,
                             'brand':brand},index=[0]
                            )

    df = df.append(dealer_df,ignore_index=True)
    df.drop_duplicates(inplace=True)

    
    
print("--- %s seconds ---" % (time.time() - start_time))
print(name)

--- 18.08899998664856 seconds ---


NameError: name 'name' is not defined

In [18]:
browser.visit(dealer_links[0])
html = browser.html
dealer_soup = BeautifulSoup(html, 'html.parser')


dealer_soup.find(class_="headline headline--uppercase headline--dark").get_text(strip=True)

'1 SOURCE TECHNICAL SERVICES, INC'

In [158]:
df


,name,street,city,state,zipcode,country,phone,lat,long,website,brand
0,"1 SOURCE TECHNICAL SERVICES, INC",11934 E. Hwy 84,ZEPHYR,Texas,76890,United States,1-844-700-3032,,,,Trane
1,1st Choice Air Solutions,1150 BLUE MOUND RD W STE 709,HASLET,Texas,76052,United States,1-844-359-2994,,,,Trane
2,1st Service Co Inc,12531 Worcester Hwy,Bishopville,Maryland,21813,United States,1-844-428-4337,,,,Trane
3,"4Seasons Air, LLC",12531 Worcester Hwy,Bishopville,Maryland,21813,United States,1-844-827-6577,,,,Trane
4,5280 Heating & Air Conditioning,16292 ORCHARD GRASS LANE,PARKER,Colorado,80134,United States,1-855-338-6722,,,,Trane
5,A-1 RELIABLE SERVICES,1282 Bandera Rd,San Antonio,Texas,78228,United States,1-855-338-6029,,,,Trane
6,A-Action Heating & Cooling,1282 Bandera Rd,San Antonio,Texas,78228,United States,1-855-338-6029,,,,Trane
7,A & A Heating & Air,11243 W US HIGHWAY 64,MURPHY,North Carolina,28906,United States,1-877-693-4420,,,,Trane
8,"A & A Precision Heating, Cooling & Refrigeration",1272 Maxwell Ave,Evansville,Indiana,47711,United States,1-855-556-5073,,,,Trane
9,A & E HEATING & A/C,505 CORNERSTONE CT,Hillsborough,North Carolina,27278,United States,1-855-257-2408,,,,Trane


In [166]:
from sqlalchemy import create_engine
import urllib
df.columns=['DealerName','PostalAddress','City','State','Zipcode','Country','Phone','LocatorLatitude','LocatorLongitude','Website','Brand']


params = urllib.parse.quote_plus('DRIVER={SQL Server Native Client 10.0};Server=gmlsql2hou; DATABASE=invplannersapp;Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
df.to_sql('GBU_Webscrape',engine,if_exists='append',index=False)

browser.quit()